In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.src.layers import TextVectorization

data = pd.read_csv('generated5000.csv')

vectorizer = TextVectorization(output_mode='int')
vectorizer.adapt(data['lawer'].tolist() + data['lawer_tp'].tolist())

# Prepare the data
source_texts = data['lawer'].tolist()
target_texts = data['lawer_tp'].tolist()

# Convert text to numerical sequences
source_sequences = vectorizer(np.array(source_texts)).numpy()
target_sequences = vectorizer(np.array(target_texts)).numpy()

# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=512, mask_zero=True),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dense(len(vectorizer.get_vocabulary()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(source_sequences, target_sequences, epochs=7)

Epoch 1/7
157/157 [==============================] - 51s 271ms/step - loss: 3.3105 - accuracy: 0.3681
Epoch 2/7
157/157 [==============================] - 42s 268ms/step - loss: 1.9160 - accuracy: 0.5805
Epoch 3/7
157/157 [==============================] - 42s 268ms/step - loss: 1.2184 - accuracy: 0.6886
Epoch 4/7
157/157 [==============================] - 42s 270ms/step - loss: 0.7187 - accuracy: 0.8220
Epoch 5/7
157/157 [==============================] - 42s 266ms/step - loss: 0.4344 - accuracy: 0.8872
Epoch 6/7
157/157 [==============================] - 42s 266ms/step - loss: 0.2871 - accuracy: 0.9195
Epoch 7/7
157/157 [==============================] - 43s 274ms/step - loss: 0.2110 - accuracy: 0.9404


In [8]:
model.save('my_model.keras')

In [10]:
new_model = tf.keras.models.load_model('my_model.keras')

In [11]:
def translate_sentence(sentence, model, vectorizer):
    sequence = vectorizer([sentence]).numpy()
    predicted_sequence = model.predict(sequence)
    predicted_sentence = ' '.join([vectorizer.get_vocabulary()[i] for i in np.argmax(predicted_sequence, axis=2)[0]])
    return predicted_sentence

In [14]:
li = ["Балаковский город суд еврейской автономной области",
     "Аннинский городской суд камчатского края",
     "Вольский межрайонный суд респ.хакасия",
     "Баргузинский областной суд г. екатеринбурга свердловской области",
     "Кировский районный суд города Екатеринбург Свердловской области"]
  
for l in li:   
    old = translate_sentence(l, model, vectorizer)
    new = translate_sentence(l, new_model, vectorizer)
    assert old == new
    print(l)
    print(old)
    print(new)
    print()


1/1 [==============================] - 0s 56ms/step
Балаковский город суд еврейской автономной области
Балаковским городским судом новгородской автономной области
Балаковским городским судом новгородской автономной области
1/1 [==============================] - 0s 45ms/step
Аннинский городской суд камчатского края
Аннинским городским судом гспб края
Аннинским городским судом гспб края
1/1 [==============================] - 0s 46ms/step
Вольский межрайонный суд респ.хакасия
Вольским межрайонным судом нижегородская
Вольским межрайонным судом нижегородская
1/1 [==============================] - 0s 50ms/step
Баргузинский областной суд г. екатеринбурга свердловской области
Баргузинским областным судом г екатеринбурга свердловской области
Баргузинским областным судом г екатеринбурга свердловской области
1/1 [==============================] - 0s 48ms/step
Кировский районный суд города Екатеринбург Свердловской области
Барабинским районным судом города тольятти свердловской области
Барабинским

In [4]:
import pickle

pickle.dump({'config': vectorizer.get_config(), 
             'weights': vectorizer.get_weights()} 
            , open("vectorizer.pkl", "wb+"))